In [1]:
def Preprocess(file):
    array = []
    for line in f:
        array.append(line)
    xc = []
    yc = []
    vertices = {}
    edges = []
    Cost = {}

    dim = int(array[0])
    for it in range(1, dim+1):
        p = array[it]
        tokens = p.split()
        xc.append(float(tokens[0]))
        yc.append(float(tokens[1]))
        vertex = (float(tokens[0]),float(tokens[1]))
        vertices[it-1] = vertex
    
    for v1 in vertices:
        for v2 in vertices:
            if(v1 != v2):
                edge = (vertices[v1],vertices[v2])
                edges.append(edge)
    
    for edge in edges:
        v1 = edge[0]
        v2 = edge[1]
        c = pow(pow(v1[0]-v2[0], 2) + pow(v1[1]-v2[1], 2), 0.5)
        Cost[edge] = c
    
    return Cost, vertices, edges

In [2]:
def InitialCandidate(vertices, edges):
    candidate = []
    for vertex in vertices:
        candidate.append(vertex)
    return candidate   

In [3]:
def Evaluate(candidate, vertices, edges, Cost):
    costval = 0
    for i in range(len(candidate)-1):
        vi1 = candidate[i]
        vi2 = candidate[i+1]
        v1 = vertices[vi1]
        v2 = vertices[vi2]
        edge = (v1,v2)
        costval += Cost[edge]
    ini = candidate[0]
    end = candidate[len(candidate) -1]
    vini = vertices[ini]
    vend = vertices[end]
    ed = (vini, vend)
    costval += Cost[ed]
    return costval      

In [4]:
def CreateNeighbor(candidate, vertices, edges, Cost):
    Neighborhood = []
    Neighborhood_Obj = {}
    Neighborhood_Move = {}
    k = 0
    for i in range(len(candidate)):
        v1 = candidate[i]
        for j in range(i+1, len(candidate)):
            #if(j!=i+1)    ### if only two non adjacent vertices are swapped
                newcandidate = []
                for vertex in candidate:
                    newcandidate.append(vertex)
                v2 = candidate[j]
                swap_move = (v1, v2)
                newcandidate[i] = v2
                newcandidate[j] = v1
                obj = Evaluate(newcandidate, vertices, edges, Cost)
                Neighborhood.append(newcandidate)
                Neighborhood_Obj[k] = obj
                Neighborhood_Move[k] = swap_move
                k += 1
    return Neighborhood, Neighborhood_Obj, Neighborhood_Move  

In [5]:
### Tabulist is a circular queue ###
class CircularQueue:
    
    def __init__(self, maxsize):
        self.maxsize = maxsize
        self.queue = [None for i in range(self.maxsize)] 
        self.front = -1
        self.rear = -1
        
        
    def enqueue(self, item):
        if((self.rear+1) % self.maxsize == self.front): ## queue full 
            return -1
        elif (self.front == -1):   ##queue is empty, 1st element is entered
            self.front = 0
            self.rear = 0
            self.queue[self.rear] = item
            return 1
        else: 
            self.rear = (self.rear+1)%self.maxsize
            self.queue[self.rear] = item
            return 1
    
    def dequeue(self):
        if(self.front == -1): ##queue empty
            return -1
        elif(self.front == self.rear): ## only one element in queue
            item = self.queue[self.front]
            self.front = -1
            self.rear = -1
            return item
        else:
            item = self.queue[self.front]
            self.front = (self.front+1)% self.maxsize
            return item
        
    def isPresent(self, item): #return True if item is present, otherwise false
        if(self.front == -1):
            return False
        elif(self.rear >= self.front):
            for i in range(self.front, self.rear+1):
                if(self.queue[i] == item):
                    return True ##if position is require return i
        elif(self.rear < self.front):
            for i in range(self.front, self.maxsize):
                if(self.queue[i] == item):
                    return True ##if position is require return i
            for i in range(0, self.rear+1):
                if(self.queue[i] == item):
                    return True ##if position is require return i
        return False

In [6]:
def BestAllowedNeighbor(candidate, vertices, edges, Cost):
    N, Nobj, Nmove =  CreateNeighbor(candidate, vertices, edges, Cost)
    best_index = -1
    bestmove = None
    best_obj = sys.maxsize
    bestneighbor = None
    global Globalobj
    global Globalopt
    for i in range(len(N)):
        neighbor = N[i]
        swap_move = Nmove[i]
        v1 = swap_move[0]
        v2 = swap_move[1]
        reverse_move = (v2,v1)
        if(Tabulist.isPresent(swap_move) != True and Tabulist.isPresent(reverse_move) != True):
            if(Nobj[i] < best_obj):
                best_index = i
                best_obj = Nobj[i]
                bestmove = swap_move
                bestneighbor = neighbor
                if(best_obj < Globalobj):
                    Globalopt = neighbor
                    Globalobj = best_obj
        else:
            if(Nobj[i] < Globalobj): ######## ASPIRATION ########
                best_index = i
                best_obj = Nobj[i]
                bestmove = swap_move
                bestneighbor = neighbor
                Globalopt = neighbor
                Globalobj = Nobj[i]
                print("Aspiration move made " + str(bestmove)+ "   "+ str(Globalopt) + "  " + str(Globalobj))
    Updatetabu(Tabulist, bestmove)
    return bestneighbor, best_obj

In [7]:
def Updatetabu(Tabulist, bestmove):
    ins = Tabulist.enqueue(bestmove)
    if(ins == -1): ## if tabulist is full, then remove an element from tabulist, then insert the item
        item = Tabulist.dequeue()
        Tabulist.enqueue(bestmove)
   

In [8]:
import string
import time
import sys

f = open("C:\\Users\\Prasangsha\\Downloads\\TSP_instance_n_20_s_1.dat")
Cost, vertices, edges = Preprocess(f)
initial = InitialCandidate(vertices, edges)
obj = Evaluate(initial, vertices, edges, Cost)
Tabutenure = len(vertices)
Tabulist = CircularQueue(Tabutenure) 
Globalobj = obj
Globalopt = initial
ite = 0

### set the number of iterations maxite
maxite = 200

while(ite < maxite):
    bn, bo = BestAllowedNeighbor(initial, vertices, edges, Cost)
    initial = bn  
    ite = ite +1
print("*******************************************************************************************")
print("The Optimal Solution is:")
for i in range(len(bn)):
    print(str(Globalopt[i]) +" ->", end = " ")
print(str(Globalopt[0]))
print("The optimal objective: "+str(Globalobj))

Aspiration move made (9, 19)   [19, 9, 4, 13, 11, 3, 18, 5, 2, 8, 10, 12, 6, 14, 15, 7, 16, 0, 17, 1]  82.85204291444069
Aspiration move made (14, 15)   [1, 19, 4, 11, 13, 3, 18, 5, 2, 8, 10, 12, 6, 15, 7, 0, 17, 16, 14, 9]  80.42318632427019
Aspiration move made (11, 13)   [1, 19, 4, 13, 11, 3, 18, 5, 2, 8, 10, 12, 6, 15, 7, 0, 17, 16, 14, 9]  79.97811143461395
Aspiration move made (16, 7)   [1, 19, 4, 13, 11, 3, 18, 5, 2, 8, 10, 12, 6, 7, 16, 0, 17, 15, 14, 9]  79.30521274204327
Aspiration move made (16, 17)   [1, 19, 4, 13, 11, 3, 18, 5, 2, 8, 10, 12, 6, 7, 17, 0, 16, 15, 14, 9]  79.13743761582316
Aspiration move made (17, 0)   [1, 19, 4, 13, 11, 3, 18, 5, 2, 8, 10, 12, 6, 7, 0, 17, 16, 15, 14, 9]  79.03647409461888
Aspiration move made (15, 14)   [19, 9, 14, 15, 16, 7, 6, 12, 10, 8, 2, 5, 18, 3, 13, 4, 11, 17, 0, 1]  78.7175244087093
Aspiration move made (13, 11)   [19, 9, 14, 15, 16, 7, 6, 12, 10, 8, 2, 5, 18, 3, 11, 4, 13, 17, 0, 1]  76.6842407967759
Aspiration move made (17, 0) 